In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
from tensorflow import keras
from tensorflow.keras.layers import Lambda
from tensorflow.keras.utils import get_custom_objects
import math
import tensorflow.keras.backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
import larq as lq

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true),axis=0))##################  axis=0

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def unbalanced_penalty_score_1out(Y_test,Y_pred) :

    s=0
    for i in range(len(Y_pred)):
        if Y_pred[i]>Y_test[i]:
            s=s+math.exp((Y_pred[i]-Y_test[i])/10)-1
        else:
            s=s+math.exp((Y_test[i]-Y_pred[i])/13)-1
    print('unbalanced_penalty_score{}'.format(s))
    return s

def error_range_1out(Y_test,Y_pred) :
    error_range=(Y_test-Y_pred).min(),(Y_test-Y_pred).max()
    print('error range{}'.format(error_range))
    return error_range

In [ ]:
segment = 3

num_test = 100

run_times = 1

nb_epochs = 5000
batch_size = 1024

patience = 100
patience_reduce_lr = 20

num_filter1 = 32
num_filter2 = 64
num_filter3 = 32

kernel1_size = 16
kernel2_size = 10
kernel3_size = 6

In [ ]:
method_name='grid_FD{}_TaNet_num_test{}_segment_{}'.format('1',num_test,segment)
# method_name='FCN_RUL_1out_train_split_test'
dataset='cmapssd'
model_name='{}_dataset_{}_log{}_time{}'.format(method_name,dataset,'x',datetime.datetime.now().strftime('%Y%m%d%H%M%S'))

In [ ]:
sequence_length=31
FD_feature_columns=[ 's2', 's3', 's4', 's6', 's7', 's8', 's9', 's11', 's12', 's13', 's14', 's15', 's17', 's20', 's21']

In [ ]:
train_feature_slice = np.load("./data_processed/train_feature.npy")
train_label_slice = np.load("./data_processed/train_label.npy")
test_feature_slice = np.load("./data_processed/test_feature.npy")
test_label_slice = np.load("./data_processed/test_label.npy")
train_feature_slice.shape, train_label_slice.shape, test_feature_slice.shape, test_label_slice.shape

In [ ]:
X_train=np.reshape(train_feature_slice,(-1,train_feature_slice.shape[1],1,train_feature_slice.shape[2]))
train_label_slice[train_label_slice>125]=125
Y_train=train_label_slice

X_train.shape, Y_train.shape

In [ ]:
X_test=np.reshape(test_feature_slice,(-1,test_feature_slice.shape[1],1,test_feature_slice.shape[2]))
test_label_slice[test_label_slice>125]=125
Y_test=test_label_slice

X_test.shape, Y_test.shape

In [ ]:
kwargs = dict(
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              # input_quantizer='weight_clip'
              )
in0 = keras.Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),
                  name='layer_20')
x1 = keras.layers.AveragePooling2D(pool_size=(int(sequence_length / segment), 1),
                                  strides=int(sequence_length / segment), name='layer_19')(in0)
x2 = keras.layers.MaxPooling2D(pool_size=(int(sequence_length / segment), 1),
                                  strides=int(sequence_length / segment), name='layer_18')(in0)
x = x1 + x2
x = keras.layers.Reshape((-1,))(x)
x = lq.layers.QuantDense(len(FD_feature_columns) // 1, use_bias=False, activation=keras.activations.relu,
                         **kwargs)(x)
kernel = lq.layers.QuantDense(len(FD_feature_columns), use_bias=False,
                              activation=keras.activations.sigmoid, name='layer_17', **kwargs)(x)
begin_senet = keras.layers.Multiply(name='layer_16')([in0, kernel])
conv0 = lq.layers.QuantConv1D(32, 3, strides=1, padding='same', dilation_rate=1, name='layer_15', **kwargs)(begin_senet)
conv0 = keras.layers.BatchNormalization()(conv0)
conv0 = keras.layers.Activation('relu', name='layer_14')(conv0)
conv0 = lq.layers.QuantConv1D(64, 3, strides=1, padding='same', dilation_rate=2, name='layer_13', **kwargs)(conv0)
conv0 = keras.layers.BatchNormalization()(conv0)
conv0 = keras.layers.Activation('relu', name='layer_12')(conv0)
conv0 = lq.layers.QuantConv1D(len(FD_feature_columns), 3, strides=1, padding='same', dilation_rate=4, name='layer_11', **kwargs)(conv0)
conv0 = keras.layers.BatchNormalization()(conv0)
conv0 = keras.layers.Activation('relu', name='layer_10')(conv0)
conv0 = conv0 + begin_senet
conv0 = lq.layers.QuantConv1D(32, 3, strides=1, padding='same', name='layer_9')(conv0)
conv0 = keras.layers.BatchNormalization()(conv0)
conv0 = keras.layers.Activation('relu', name='layer_8')(conv0)
conv0 = lq.layers.QuantConv1D(64, 3, strides=1, padding='same', name='layer_7')(conv0)
conv0 = keras.layers.BatchNormalization()(conv0)
conv0 = keras.layers.Activation('relu', name='layer_6')(conv0)
conv0 = lq.layers.QuantConv1D(32, 3, strides=1, padding='same', name='layer_5')(conv0)
conv0 = keras.layers.BatchNormalization()(conv0)
conv0 = keras.layers.Activation('relu', name='layer_4')(conv0)
conv0 = keras.layers.GlobalAveragePooling2D(name='layer_3')(conv0)
conv0 = lq.layers.QuantDense(16, activation='relu', name='layer_2')(conv0)
out = lq.layers.QuantDense(1, activation='relu', name='layer_1')(conv0)
model = keras.models.Model(inputs=in0, outputs=[out])

In [ ]:
lq.models.summary(model)

In [ ]:
optimizer = keras.optimizers.Adam()

In [ ]:
model.compile(loss='mse',#loss=root_mean_squared_error,
              optimizer=optimizer,
              metrics=[root_mean_squared_error])

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                  patience=patience_reduce_lr, min_lr=0.0001)

In [ ]:
earlystopping=keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error',patience=patience,verbose=1)
modelcheckpoint=keras.callbacks.ModelCheckpoint(monitor='val_root_mean_squared_error',filepath="./model/FCN_RUL_1out_train_valid_test/{}.h5".format(model_name),save_best_only=True,verbose=1)

In [ ]:
hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
          verbose=1, validation_data=(X_test, Y_test), callbacks = [reduce_lr,earlystopping,modelcheckpoint])

In [ ]:
print(hist.history.keys())
epochs=range(len(hist.history['loss']))
plt.figure()
plt.plot(epochs,hist.history['loss'],'b',label='Training loss')
plt.plot(epochs,hist.history['val_loss'],'r',label='Validation val_loss')
plt.title('Traing and Validation loss')
plt.legend()
plt.show()

In [ ]:
model=keras.models.load_model("./model/FCN_RUL_1out_train_valid_test/{}.h5".format(model_name),custom_objects={'root_mean_squared_error': root_mean_squared_error})
for layer in model.layers:
    layer.trainable=False

Y_pred=model.predict(X_test)
Y_pred = np.where(Y_pred > 125, 125, Y_pred)
rmse_value=rmse(Y_test,Y_pred)
print('rmse:{}'.format(rmse_value))

In [ ]:
unbalanced_penalty_score=unbalanced_penalty_score_1out(Y_test,Y_pred)
error_range=error_range_1out(Y_test,Y_pred)

In [ ]:
pred, real = [], []
for i in range(len(Y_test)-1):
    if Y_test[i+1] > Y_test[i]:
        pred.append(Y_pred[i])
        real.append(Y_test[i])
pred = np.array(pred)
real = np.array(real)
unbalanced_penalty_score_1out(pred,real)

# 画图

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

config = {
            "font.family": 'serif',
            "font.size": 20,# 相当于小四大小
            "mathtext.fontset": 'stix',#matplotlib渲染数学字体时使用的字体，和Times New Roman差别不大
            "font.serif": ['Times New Roman'],#宋体
            'axes.unicode_minus': False # 处理负号，即-号
         }
rcParams.update(config)

In [ ]:
np.savetxt('FD001_pred.txt',pred,fmt='%d',delimiter=None)
np.savetxt('FD001_real.txt',real,fmt='%d',delimiter=None)

In [ ]:
fig, ax = plt.subplots(figsize=(8,2))
plt.plot(pred, color='#156CB3', label='pred_RUL')
plt.plot(real, color='#F49F05', label='real_RUL')
plt.grid()
plt.legend()
plt.title('FD001')
plt.xlabel("Engine No")
plt.ylabel("RUL")
# plt.savefig("D:/workspace/paper/1/figs/FD001_res.png", bbox_inches='tight',dpi=600)

In [ ]:
np.save('./size.npy', res)

In [ ]:
res = np.load('./size.npy')
res = np.where(res > 115, 115, res)
labels = ['3.72Mb', '2.15Mb', '0.12Mb']
colors = ['#156CB3','#F49F05','#03A64A']
for i in range(len(res)):
    plt.plot(res[i][4522:4794],'.-', color=colors[i],label=labels[i])
    plt.title("FD001_#42")
    plt.grid()
    plt.xlabel("Cycles")
    plt.ylabel("RUL")
plt.plot(Y_test[4522:4794], label='real_RUL', color='#D92525')
plt.xticks(fontsize="20")
plt.yticks(fontsize="20")
plt.legend(loc='lower left', fontsize="20")
plt.xlabel('Cycles', fontsize="20")
plt.ylabel('RUL', fontsize="20")
plt.title('FD001_#42',fontsize="20")
# plt.grid()
plt.savefig("D:/workspace/paper/1/figs/FD001_#42_size.png", bbox_inches='tight',dpi=600)

In [ ]:
s = np.load('./size.npy')
res.append(s[-1])
np.save('./opt.npy', res)

In [ ]:
res = np.load('./opt.npy')+10
res = np.where(res > 125, 125, res)
labels = ['No_optimizer', 'BCTA_only', 'BCTA+BTCN']
colors = ['#156CB3','#F49F05','#03A64A']
for i in range(len(res)):
    plt.plot(res[i][4522:4794],'.-', color=colors[i],label=labels[i])
    plt.title("FD001_#42")
    plt.grid()
    plt.xlabel("Cycles")
    plt.ylabel("RUL")
plt.plot(Y_test[4522:4794], label='real_RUL', color='#D92525')
plt.xticks(fontsize="20")
plt.yticks(fontsize="20")
plt.legend(loc='lower left', fontsize="20")
plt.xlabel('Cycles', fontsize="20")
plt.ylabel('RUL', fontsize="20")
plt.title('FD001_#42',fontsize="20")
plt.savefig("D:/workspace/paper/1/figs/FD001_#42_opt_125.png", bbox_inches='tight',dpi=600)

In [ ]:
representation_model = tf.keras.Model(inputs=model.inputs, outputs=model.get_layer('layer_17').output)

In [ ]:
cycles = []
for i in range(len(Y_test)-1):
    if Y_test[i] > Y_test[i-1]:
        cycles.append(i)
len(cycles)

In [ ]:
res = representation_model.predict(X_train)
data = res.mean(axis=0).reshape(15,1)

In [ ]:
s = np.where(res >= 0.3, 1, 0)
data = []
for i in range(len(cycles)-1):
    cur = s[cycles[i]:cycles[i+1],:]
    data.append(cur.mean(axis=0))
data = np.squeeze(np.array(data))

In [ ]:
np.savetxt('example.csv', data, delimiter=',')

In [ ]:
plt.figure(figsize=(16, 4))
sns.boxplot(data=data, palette="Blues")
plt.grid()

In [ ]:
data = np.array([[9.7551495e-02],
       [2.2458841e-01],
       [3.2795912e-01],
       [8.7800506e-04],
       [1.5283097e-01],
       [6.8548906e-01],
       [1.5941867e-01],
       [9.7184944e-01],
       [4.2998475e-01],
       [7.5283670e-01],
       [2.5214184e-02],
       [9.4165720e-02],
       [4.8141783e-01],
       [8.1823915e-01],
       [1.3626677e-01]])

In [ ]:
data = np.array([0 if i < 0.3 else 1 for i in data]).reshape(15,1)

In [ ]:
fig, ax = plt.subplots(figsize=(2,4))
heatmap = sns.heatmap(data, cbar=True, ax=ax, annot=True, fmt='.2f', cmap="Blues", annot_kws={'rotation': 0})
cbar = heatmap.collections[0].colorbar
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=90)
cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(), rotation=90, va='center')
plt.yticks([i + 0.5 for i in range(len(FD_feature_columns))], FD_feature_columns, rotation=0)
plt.tight_layout()
plt.title("FD001")
# plt.savefig("D:/workspace/paper/1/figs/test_features_weight_FD001_#2.png",dpi=600, bbox_inches='tight')